## exploration

### paramètres pandas

In [ ]:
pd.set_option('display.max_columns', 184)
pd.set_option('display.max_rows', 184)
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_seq_items', 184)
pd.set_option('display.width', 100)

### utilitaires

In [ ]:
class Chrono(object):  
    def __init__(self):
        self.start()
    def start(self):  
        if hasattr(self, 'interval'):  
            del self.interval  
        self.start_time = time.time()  
  
    def stop(self):  
        if hasattr(self, 'start_time'):  
            self.interval = time.time() - self.start_time  
            del self.start_time # Force timer reinit  
            print('{h}:%.3f temps passé'.format(h = int(self.interval//60)) % (self.interval%60))

### nettoyage

In [ ]:
def netoyageCat(serie, seuil=20, new_val="Autre", fillna=True):
    '''
    sur une série du df on retire les valeurs qui passe sous le seuil d'occurence pour les remplasser par la valeurs générique new_val
    '''
    mask_bool = serie.value_counts() < seuil
    mask_autre = mask_bool.copy()
    mask_autre[mask_bool] = new_val
    serie = serie.replace(mask_autre[mask_bool])
    if fillna:
        serie.fillna(new_val, inplace=True)
    return serie

In [ ]:
def multiOnehotTags(serie, seuil=10, en='', clean=True):
    '''
    onehot pour une variable qui à plusieur valeurs dans une celule
    '''
    liste = listMainCat(serie=serie, seuil=seuil, en=en, clean=clean)
    df_temp = pd.DataFrame(columns=liste.keys())
    for i in serie.index:
        val = serie.loc[i]
        if type(val) == str:
            for r in range(val.count(','+ en)+1):
                virgule = val.find(',' + en)
                if virgule == -1:
                    if val[len(en):] in liste:
                        df_temp.loc[i,val[len(en):]] = 1
                    else:
                        df_temp.loc[i] = 0
                else:
                    if val[len(en):virgule] in liste:
                        df_temp.loc[i,val[len(en):virgule]] = 1
                    else:
                        df_temp.loc[i] = 0
                    val = val[virgule + 1:]
        else:
            df_temp.loc[i] = 0
    return df_temp.fillna(0)

In [ ]:
def dropOnehot(df, seuil):
    '''
    éjecte les colonnes qui sont remplie en dessous du seuil choisis
    seuil peut être un entier et du coup il ejecte ceux qui sont moins que ce nombre
    ou il peut être une fraction qui est le pourcentage en desous du quel il eject
    éject aussi les one hot plein à 100%
    '''
    r=0
    liste_drop = []
    if isinstance(seuil, int):
        for i in range(df.shape[1]):
            if (df.iloc[:,i].value_counts()[1.0] < seuil) | (df.iloc[:,i].value_counts()[1.0]/df.shape[0] == 1):
                liste_drop.append(df.columns[i])
                r+=1
    elif isinstance(seuil, float):
        for i in range(df.shape[1]):
            if (df.iloc[:,i].value_counts()[1.0]/df.shape[0] < seuil) | (df.iloc[:,i].value_counts()[1.0]/df.shape[0] == 1):
                liste_drop.append(df.columns[i])
                r+=1
    print(r,' colonnes ejecte dans ', df.shape[1] ,' colonnes')
    return df.drop(liste_drop, axis=1)

In [ ]:
# fait un Onehot à la mano
def Onehot(serie):
    load=0
    df_temp = pd.DataFrame()
    for i in range(serie.shape[0]):
        val = serie.iloc[i]
        if type(val) == str:
            df_temp.loc[i,val] = 1
        else:
            df_temp.loc[i,:] = 0
        # bar de progression
        if load < i//1000:
            load = i//1000
            clear_output(wait=True)
            print(i, '/', serie.shape[0], '(%.4f)' % (i/serie.shape[0]))
    df_temp.fillna(0, inplace=True)
    return df_temp

In [1]:
def listMainCat(serie, seuil=10, en='', clean=True):
    '''
    liste dans la série toutes les valeurs existantes et trie du plus au moins frequent (ne considère pas ceux en dessous du seuil)
    seuil peut être un entier et du coup il ejecte ceux qui sont moins que ce nombre
    ou il peut être une fraction qui est le pourcentage en desous duquel il ejecte
    clean permet de retirer les éspaces qui sont au début ou à la fin des chaines de caractères
    '''
    serie_temp = pd.Series()
    serie_values = serie.value_counts()
    for i in range(serie_values.shape[0]):
        val = serie_values.index[i]
        for r in range(val.count(',')+1):
            if clean:
                val = val.strip()
            virgule = val.find(',')
            if virgule == -1:
                if val[len(en):] in serie_temp:
                    serie_temp[val[len(en):]] += serie_values[i]
                else:
                    serie_temp[val[len(en):]] = serie_values[i]
            else:
                if val[len(en):virgule] in serie_temp:
                    serie_temp[val[len(en):virgule]] += serie_values[i]
                else:
                    serie_temp[val[len(en):virgule]] = serie_values[i]
                val = val[virgule+1:]
    serie_temp.sort_values(ascending=False, inplace=True)
    if isinstance(seuil, int):
        for i in range(serie_temp.shape[0]):
            if serie_temp.iloc[i] < seuil:
                serie_temp = serie_temp.iloc[0:i]
                break
    elif isinstance(seuil, float):
        for i in range(serie_temp.shape[0]):
            if serie_temp.iloc[i]/serie.shape[0] < seuil:
                serie_temp = serie_temp.iloc[0:i]
                break
    return serie_temp

In [ ]:
def post_to_words( post_text, stopwords_lem=True):
    # 1. Retire les caractères spéciaux et certains paternes non souhaités
    post_text = re.sub(r"(\\n)|(\.\\n)|(\'\w+)|(http*\S+)|[^\w\s\.,#+-]", " ", post_text)
    post_text = re.sub(r"(\.\s)|(\.$)|(,\s)|(\s#\s)|(\'\w+)|(\s\-\s)", " ", post_text)
    post_text = re.sub(r"(\s-?\+?\d{0,4}\s)", " ", post_text)
    post_text = re.sub(r"\s+", " ", post_text)
    # 2. passe tout en minuscule, et découpe les mots
    words = post_text.lower().split()
    if stopwords_lem :
        # 3. Retire les Stop Words
        words = [w for w in words if not w in stops]
        # 4. Garde le radical des mots
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(w) for w in words]
    return " ".join( words )

### visualisation

In [ ]:
# retourne le tableau de contingence
def continTab(serieliste1, liste1, serieliste2, liste2, normalize=False):
    reste = pd.Series()
    df_cont = pd.DataFrame()
    for i in range(liste1.shape[0]): # colonne
        for r in range(liste2.shape[0]): # lignes
            df_cont.loc[liste2.index[r], liste1.index[i]] = serieliste2[serieliste2.str.contains(liste2.index[r],na=False, regex=False) & serieliste1.str.contains(liste1.index[i],na=False, regex=False)].shape[0]
        clear_output(wait=True)
        print(i+1, '/', liste1.shape[0])
        df_cont.loc['Le Reste',liste1.index[i]] = liste1[i] - df_cont[liste1.index[i]].sum()
        if normalize:
            df_cont = df_cont/df_cont[liste1.index[i]].sum()
        df_cont.loc['Total',liste1.index[i]] = df_cont[liste1.index[i]].sum()
        reste = serieliste1.str.contains(liste1.index[i],na=False, regex=False) | reste
    for r in range(liste2.shape[0]):
        df_cont.loc[liste2.index[r],'Le Reste'] = serieliste2[serieliste2.str.contains(liste2.index[r],na=False, regex=False) & ~reste].shape[0]
    if normalize:
        df_cont['Le Reste'] = df_cont['Le Reste']/df_cont['Le Reste'].sum()
    df_cont['Total'] = df_cont.sum(axis=1)
    if df_cont.loc['Le Reste'].sum() == 0:
        df_cont.drop(['Le Reste'], axis=0, inplace=True)
    if df_cont.loc[:,'Le Reste'].sum() == 0:
        df_cont.drop(['Le Reste'], axis=1, inplace=True)
    return df_cont

In [ ]:
def correspTab(serielisteC, liste1, serielisteR, liste2, annot=False):
    cont = continTab(serielisteC, liste1, serielisteR, liste2)
    indep = cont.loc[:,["Total"]].dot(cont.loc[["Total"],:])/len(serielisteC)
    measure = (cont-indep)**2/indep
    xi_n = measure.sum().sum()
    print(xi_n)
    table = measure/xi_n
    plt.figure(figsize=(.6*len(liste1),.6*len(liste2)))
    if annot:
        sns.heatmap(table.iloc[:-1,:-1], annot=cont.iloc[:-1,:-1], linewidths=.2)
    else:
        sns.heatmap(table.iloc[:-1,:-1], linewidths=.2)
    plt.show()
    return table, cont

In [ ]:
def listBarPlot(serie, serieliste, liste, n_plot=10, moy=False, Autre=False):
    if liste.shape[0] < n_plot:
        n_plot = liste.shape[0]
    reste = pd.Series()
    plt.figure(figsize=(30,7*((n_plot//4)+1)))
    for i in range(n_plot):
        plt.subplot((n_plot//4)+1, 4,i+1)
        val_counts = serie[serieliste.str.contains(liste.index[i],na=False, regex=False)].value_counts()
        if moy:
            val_counts.plot.bar(title='{titre} (moyenne: {moy})'.format(titre = liste.index[i], moy = (val_counts.index * val_counts / val_counts.sum()).sum()))
        else:
            val_counts.plot.bar(title=liste.index[i])
        reste = serieliste.str.contains(liste.index[i],na=False, regex=False) | reste
        plt.legend()
    if Autre:
        plt.subplot((n_plot//4)+1, 4,i+2)
        if moy:
            serie[~reste].value_counts().plot.bar(title='Le Reste (moyenne: {moy})'.format(moy = (serie[~reste].value_counts().index * serie[~reste].value_counts() / serie[~reste].value_counts().sum()).sum()))
        else:
            serie[~reste].value_counts().plot.bar(title='Le Reste')
        plt.legend()
    plt.show()

In [ ]:
def biListBarPlot(serieliste1, liste1, serieliste2, liste2, n_plot=10, n_cat=10,Autre=False):
    if liste2.shape[0] < n_plot:
        n_plot = liste2.shape[0]
    reste = pd.Series()
    plt.figure(figsize=(40,7*((n_plot//4)+1)))
    for i in range(n_plot):
        plt.subplot((n_plot//4)+1, 4, i+1)
        serie_sum = pd.Series()
        for r in range(n_cat):
            serie_sum[liste1.index[r]] = serieliste1[serieliste1.str.contains(liste1.index[r],na=False, regex=False) & serieliste2.str.contains(liste2.index[i],na=False, regex=False)].shape[0]
        serie_sum['Le Reste'] = liste2[i] - serie_sum.sum()
        serie_sum.plot.bar(title=liste2.index[i])
        reste = serieliste2.str.contains(liste2.index[i],na=False, regex=False) | reste
        plt.legend()
    if Autre:
        plt.subplot((n_plot//4)+1, 4,i+2)
        serie_sum = pd.Series()
        for r in range(n_cat):
            serie_sum[liste1.index[r]] = serieliste1[serieliste1.str.contains(liste1.index[r],na=False, regex=False) & reste].shape[0]
        serie_sum.plot.bar(title='Le Reste')
        plt.legend()
    plt.show()

In [ ]:
def scatter_hist(x, y, ax, ax_histx, ax_histy):
    # no labels
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)

    # the scatter plot:
    ax.scatter(x, y)

    # now determine nice limits by hand:
    binwidth = 0.25
    xymax = max(np.max(np.abs(x)), np.max(np.abs(y)))
    lim = (int(xymax/binwidth) + 1) * binwidth

    bins = np.arange(-lim, lim + binwidth, binwidth)
    ax_histx.hist(x, bins=bins)
    ax_histy.hist(y, bins=bins, orientation='horizontal')

In [ ]:
def add_identity(axes, *line_args, **line_kwargs):
    identity, = axes.plot([], [], *line_args, **line_kwargs)
    def callback(axes):
        low_x, high_x = axes.get_xlim()
        low_y, high_y = axes.get_ylim()
        low = max(low_x, low_y)
        high = min(high_x, high_y)
        identity.set_data([low, high], [low, high])
    callback(axes)
    axes.callbacks.connect('xlim_changed', callback)
    axes.callbacks.connect('ylim_changed', callback)
    return axes

In [ ]:
def perf(model, std=False, exp=False):
    '''
    fait un graphique des performances du modèle par rapport aux vraies données
    '''
    plt.figure(figsize=(10,10))
    ax = plt.subplot()
    if std:
        ax.scatter(model.predict(X_test_std), y_test)
    elif exp:
        ax.scatter(np.exp(model.predict(X_test)), y_test)
    else:
        ax.scatter(model.predict(X_test), y_test)
    add_identity(ax, color='r', ls='--')
    ax.set_aspect('equal')
    plt.show
    if std:
        print('R2 : %3f'%(metrics.r2_score(y_test, model.predict(X_test_std))))
    elif exp:
        print('R2 : %3f'%(metrics.r2_score(y_test, np.exp(model.predict(X_test)))))
    else:
        print('R2 : %3f'%(metrics.r2_score(y_test, model.predict(X_test))))

#### NLP

In [ ]:
def plot_top_words(model, feature_names, n_top_words, nb_line_plot, title):
    '''
    c'est pour les NMF
    '''
    rows = int(nb_line_plot)
    fig, axes = plt.subplots(rows, 6, 
                             figsize=(30, rows*10), 
                             sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        if(topic_idx < nb_line_plot*6):
            top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
            top_features = [feature_names[i] for i in top_features_ind]
            weights = topic[top_features_ind]

            ax = axes[topic_idx]
            bartopic = ax.barh(top_features, weights, height=0.7)
            bartopic[0].set_color('#f48023')
            ax.set_title(f'Topic {topic_idx +1}',
                         fontdict={'fontsize': 30})
            ax.invert_yaxis()
            ax.tick_params(axis='both', which='major', labelsize=20)
            for i in 'top right left'.split():
                ax.spines[i].set_visible(False)
            fig.suptitle(title, fontsize=36)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

## Modelisation

In [ ]:
class ModelClass:
    """
    Permet de manipuler différents modeles de Classification
    (supervisé et non supervisé)
    dans le même objet pour simplifier les comparaisons et visualisations
    """
    def __init__(self, y_train: np.ndarray, y_test: np.ndarray, scoring: Optional[str] = None, prop_tags: Union[str, int] = 'origin'):
        self.models = pd.DataFrame(columns=[
            'name',
            'data_use',
            'model',
            'confusion_matrice',
            'Error',
            'Jaccard Macro',
            'Precision Macro',
            'Recall Macro',
            'F_mesure Macro',
            'auc Macro',
            'Jaccard Micro',
            'Precision Micro',
            'Recall Micro',
            'F_mesure Micro',
            'auc Micro',
            'proportion_de_prédiction',
            'fit_time',
            'pred_time',
            'y_pred',
            'best_params',
            'grid_params',
            'grid_score',
            'params'
        ])
        self.scoring = scoring
        self.prop_tags = prop_tags
        self.models.set_index('name', inplace=True)
        self.y_train = y_train
        self.y_test = y_test
        self.x = {}
    def save_data(self, path: str):
        self.models.drop(['model'], axis=1).loc[self.models['y_pred'].notna()].to_pickle(path=path)
        print('données du fichier '+ path + ' bien sauvegardé')
    def append_data(self, path: str):
        new_data = pd.read_pickle(path)
        self.models = self.models.append(new_data)
        print('données du fichier '+ path + ' bien chargé')
    def add_model(self, name: str, model, params=None):
        """
        Effectue une recherche par grille sur les paramètres.
        """
        self.models.loc[name, 'data_use'] = 'no fit'
        self.models.loc[name]['model'] = model
        self.models.loc[name]['grid_params'] = params
    def add_prediction(self, name: str, y_pred: np.ndarray, fit_time: float = 0, pred_time: float = 0):
        """
        ajout des données prédites (utilisé pour les modeles non supervisés).
        """
        self.models.loc[name, 'fit_time'] = fit_time
        self.models.loc[name, 'pred_time'] = pred_time
        self.models.loc[name]['y_pred'] = y_pred
    def add_data(self, name: str, x_train, x_test):
        self.x[name] = {
            'x_train': x_train,
            'x_test': x_test
        }
    def fit(self, name: Union[str, list] = 'all', data_name=None, cv: Optional[int] = None):
        if name == 'all':
            name = self.models.index.to_list()
        elif type(name) == str:
            print('préparation du modele ' + str(name))
            self._fit_process(self.models.loc[name], data_name, cv)
        elif type(name) == list:
            print('préparation des modeles ' + ' '.join(name))
            for model_name in name:
                self._fit_process(self.models.loc[model_name], data_name, cv)
    def _fit_process(self, model: pd.Series, data_name: str, cv: Optional[int]):
        if data_name == None:
            data_name = list(self.x.keys())[-1]
        x_train = self.x[data_name]['x_train']
        x_test = self.x[data_name]['x_test']
        if model['grid_params'] != None:
            grid = GridSearchCV(model['model'], model['grid_params'], cv=cv, scoring=self.scoring, verbose=3)
            grid.fit(x_train, self.y_train)
            model['grid_score'] = {
                'mean_test_score': grid.cv_results_['mean_test_score'],
                'mean_fit_time': grid.cv_results_['mean_fit_time'],
                'mean_score_time': grid.cv_results_['mean_score_time'],
                'params': grid.cv_results_['params']
            }
            model['model'] = grid.best_estimator_
        start = time.time()
        model['model'].fit(x_train, self.y_train)
        stop = time.time()
        print('le model ' + str(model.name) + ' à fini son entrainement')
        model['fit_time'] = stop - start
        model['best_params'] = model['model'].get_params()
        model['data_use'] = data_name
        start = time.time()
        y_pred = np.array(model['model'].predict_proba(x_test))
        stop = time.time()
        model['pred_time'] = stop - start
        if len(y_pred.shape) == 3:
            model['y_pred'] = y_pred[:,:,1].T
        elif len(y_pred.shape) == 2:
            model['y_pred'] = y_pred
        self.compute_perf(str(model.name))
        print('tout est bon pour le modele ' + str(model.name))
    def compute_perf(self, model_name: str, prop_tags: Optional[Union[str, int]] = None):
        model: pd.Series = self.models.loc[model_name]
        y_true: np.ndarray = self.y_test
        y_pred: np.ndarray = model['y_pred']
        n_classes = y_true.shape[1]
        n_indiv = y_true.shape[0]
        if not prop_tags :
            prop_tags = self.prop_tags
        if prop_tags == 'origin':
            prop_tags = y_true.sum()/n_indiv
        i = 0.5
        seuil = 1
        while i > 0.005 :
            if (y_pred >= seuil).sum()/n_indiv < prop_tags :
                seuil -= i
            else :
                seuil += i
            i /= 2
        y_predict = y_pred >= seuil
        print('le seuil est: '+ str(seuil))
        mcm = metrics.multilabel_confusion_matrix(y_true, y_predict)
        tn = mcm[:, 0, 0].mean()
        tp = mcm[:, 1, 1].mean()
        fn = mcm[:, 1, 0].mean()
        fp = mcm[:, 0, 1].mean()
        model['confusion_matrice'] = [[tn, fn], [fp, tp]]
        model['Error'] = metrics.hamming_loss(y_true, y_predict)
        model['Jaccard Macro'] = metrics.jaccard_score(y_true, y_predict, average='macro')
        model['Precision Macro'] = metrics.precision_score(y_true, y_predict, average='macro')
        model['Recall Macro'] = metrics.recall_score(y_true, y_predict, average='macro')
        model['F_mesure Macro'] = metrics.f1_score(y_true, y_predict, average='macro')
        model['Jaccard Micro'] = metrics.jaccard_score(y_true, y_predict, average='micro')
        model['Precision Micro'] = metrics.precision_score(y_true, y_predict, average='micro')
        model['Recall Micro'] = metrics.recall_score(y_true, y_predict, average='micro')
        model['F_mesure Micro'] = metrics.f1_score(y_true, y_predict, average='micro')
        model['proportion_de_prédiction'] = (tp + fp)/(tp + fn)
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = metrics.roc_curve(y_true[:, i], y_pred[:, i])
            if y_true[:, i].sum() == 0:
                tpr[i] = fpr[i]
            roc_auc[i] = metrics.auc(fpr[i], tpr[i])
        precision, recall, _ = metrics.precision_recall_curve(y_true.ravel(), y_pred.ravel())
        
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = metrics.roc_curve(y_true.ravel(), y_pred.ravel())
        roc_auc["micro"] = metrics.auc(fpr["micro"], tpr["micro"])
        
        # First aggregate all false positive rates
        all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
        
        # Then interpolate all ROC curves at this points
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(n_classes):
            mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

        # Finally average it and compute AUC
        mean_tpr /= n_classes
            
        fpr["macro"] = all_fpr
        tpr["macro"] = mean_tpr
        roc_auc["macro"] = metrics.auc(fpr["macro"], tpr["macro"])
        
        model['auc Micro'] = roc_auc["micro"]
        model['auc Macro'] = roc_auc["macro"]
        model['params'] = {
            'fpr': fpr,
            'tpr': tpr,
            'roc_auc': roc_auc,
            'precision': precision,
            'recall': recall
        }
    def plot_roc(self, model_list: list = [], plot_size: int = 10):
        if model_list == []:
            model_list = self.models.index.to_list()
        lenth = len(model_list)
        sqrt = math.floor(math.sqrt(lenth))
        plt.figure(figsize=(plot_size*sqrt,plot_size*math.ceil(lenth/sqrt)))
        for it, name in enumerate(model_list):
            plt.subplot(math.ceil(lenth/sqrt), sqrt, it+1)
            model = self.models.loc[name]
            fpr = model['params']['fpr']
            tpr = model['params']['tpr']
            roc_auc = model['params']['roc_auc']
            plt.plot(
                fpr["micro"],
                tpr["micro"],
                label="micro-average Courbe ROC (surface = {0:0.2f})".format(roc_auc["micro"])
            )
            plt.plot(
                fpr["macro"],
                tpr["macro"],
                label="macro-average Courbe ROC (surface = {0:0.2f})".format(roc_auc["macro"])
            )
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xticks(fontsize=14)
            plt.yticks(fontsize=14)
            plt.xlabel("Ratio Faux Positif", fontsize=14)
            plt.ylabel("Ratio Vrais Positif", fontsize=14)
            plt.title("Courbe ROC du model : "+ name, fontsize=16)
            plt.legend(loc="lower right", fontsize=12)
        plt.show()
    def plot_precision_recall(self, model_list: list = [], plot_size: int = 10):
        if model_list == []:
            model_list = self.models.index.to_list()
        lenth = len(model_list)
        sqrt = math.floor(math.sqrt(lenth))
        plt.figure(figsize=(plot_size*sqrt,plot_size*math.ceil(lenth/sqrt)))
        for it, name in enumerate(model_list):
            plt.subplot(math.ceil(lenth/sqrt), sqrt, it+1)
            model = self.models.loc[name]
            precision = model['params']['precision']
            recall = model['params']['recall']
            plt.plot(
                precision,
                recall
            )
            plt.xticks(fontsize=14)
            plt.yticks(fontsize=14)
            plt.xlabel("Précision", fontsize=14)
            plt.ylabel("Recall", fontsize=14)
            plt.title("Courbe Précision Recall du model : "+ name, fontsize=16)
        plt.show()
    def plot_perf(self, perf_list: List[str] = [], model_list: List = [], plot_size: int = 10):
        if perf_list == []:
            perf_list = ['Error', 'Jaccard Macro', 'Precision Macro',
                        'Recall Macro', 'F_mesure Macro', 'auc Macro',
                        'Jaccard Micro', 'Precision Micro', 'Recall Micro',
                        'F_mesure Micro', 'auc Micro',
                        'fit_time', 'pred_time']
        if model_list == []:
            models = self.models
        else:
            models = self.models.loc[model_list]
        lenth = len(perf_list)
        sqrt = math.floor(math.sqrt(lenth))
        plt.figure(figsize=(plot_size*sqrt,plot_size*math.ceil(lenth/sqrt)))
        for it, score in enumerate(perf_list):
            plt.subplot(math.ceil(lenth/sqrt), sqrt, it+1)
            value = models[score].values.astype('float')
            colors = cm.winter(value / float(max(value)))
            plt.bar(models.index, value, color = colors)
            plt.title('Score '+score+' par Modele', fontsize=18)
            plt.xticks(rotation=60, fontsize=14)
            plt.yticks(fontsize=16)
            plt.xlabel('')
            plt.ylabel(score, fontsize=14)
        plt.tight_layout()
        plt.show()
    def plot_grid(self, model_name: str, plot_height: int = 10, model_lenth: float = 1.5):
        model = self.models.loc[model_name]
        params = []
        val = ''
        for param_list in model['grid_score']['params']:
            for param in param_list.keys():
                val+= str(param) + ': ' +str(param_list[param]) + '\n'
            params.append(val)
            val = ''
        plt.figure(figsize=(model_lenth*len(params), plot_height*3))
        plt.suptitle('Grille de paramètre pour '+ model_name, fontsize=22)
        plt.subplot(311)
        value = model['grid_score']['mean_test_score']
        colors = cm.winter(value / float(max(value)))
        plt.bar(params, value, color = colors)
        plt.title('Score moyen par paramètre', fontsize=18)
        plt.xticks(rotation=30, fontsize=14)
        plt.yticks(fontsize=16)
        plt.xlabel('')
        plt.ylabel('score moyen des folds', fontsize=14)
        
        plt.subplot(312)
        value = model['grid_score']['mean_fit_time']
        colors = cm.winter(value / float(max(value)))
        plt.bar(params, value, color = colors)
        plt.title('temp d\'entrainement par paramètre', fontsize=18)
        plt.xticks(rotation=30, fontsize=14)
        plt.yticks(fontsize=16)
        plt.xlabel('')
        plt.ylabel('score moyen des folds', fontsize=14)
        
        plt.subplot(313)
        value = model['grid_score']['mean_score_time']
        colors = cm.winter(value / float(max(value)))
        plt.bar(params, value, color = colors)
        plt.title('temps de prédiction par paramètre', fontsize=18)
        plt.xticks(rotation=30, fontsize=14)
        plt.yticks(fontsize=16)
        plt.xlabel('')
        plt.ylabel('score moyen des folds', fontsize=14)
        plt.tight_layout(rect=[0, 0.03, 1, 0.98])
        plt.show

In [ ]:
from sklearn.model_selection import GridSearchCV
import timeit
from sklearn import metrics
import math

class ModelRegres:
    def __init__(self):
        self.result = pd.DataFrame(columns=[
            'name',
            'params',
            'trainError',
            'testError R2',
            'testError RMSE',
            'learningTime',
            'predictTime'
        ])
        self.best_models = []
    def err(self, model, X, y, scoring):
        """
        Calcul de l'erreur du les prédictions du model
        """
        if scoring == "r2":
            return metrics.r2_score(y, model.predict(X))
        elif scoring == 'neg_root_mean_squared_error':
            return math.sqrt(np.mean((model.predict(X) - y) ** 2))
        else:
            print('scoring invalide')
    def search(self, name, model, params, data, folds=5, verbose=1, scoring="r2"):
        """
        Effectue une recherche par grille sur les paramètres.
        """
        if data:
            X_train = data['X_train']
            y_train = data['y_train']
            X_test = data['X_test']
            y_test = data['y_test']
        
        search = GridSearchCV(model, params, scoring=scoring, verbose=verbose, cv=folds, n_jobs=-1)
        search.fit(X_train, y_train)
        
        self.result = self.result.append({
            'name': name,
            'params': search.best_params_,
            'trainError': self.err(search.best_estimator_, X_train, y_train, scoring),
            'testError R2': self.err(search.best_estimator_, X_test, y_test, 'r2'),
            'testError RMSE': self.err(search.best_estimator_, X_test, y_test, 'neg_root_mean_squared_error'),
            'learningTime': search.refit_time_,
            'predictTime': 0
        }, ignore_index=True)
        
        self.best_models.append(search.best_estimator_)
        print('Résultat ', name,' :')
        print('Paramètres : ', search.best_params_)
        print('Erreur sur le dataset d\'entraînement : ', self.err(search.best_estimator_, X_train, y_train, scoring))
        print('Erreur sur le dataset de test : ', self.err(search.best_estimator_, X_test, y_test, scoring))
        print('Temps d\'apprentissage : ', search.refit_time_)
        
        return search.cv_results_
    def plot(self, name_model, grid, param1, name1, param2, name2):
        try:
            param2
            name2
            test = 1
        except:
            test = 2
        if test == 1:
            fig, ax = plt.subplots(1,1)
            img = ax.imshow(np.log(-np.array(grid['mean_test_score']).reshape(len(param1), len(param2)) + grid['mean_test_score'].max()))
            ax.set_xticklabels(['{:.2e}'.format(x) for x in param2] , Rotation=45, fontdict={
            'horizontalalignment': 'right'})
            ax.set_yticklabels(['{:.2e}'.format(x) for x in param1])
            plt.title(name_model, fontsize =16)
            plt.ylabel(name1, fontsize =14)
            plt.xlabel(name2, fontsize =14)
            plt.show

## Non supervisé

### ACP

In [ ]:
def display_scree_plot(pca):
    '''
    visualiser la progression de l'inertie par l'ajout de nouvelle composantes
    '''
    scree = pca.explained_variance_ratio_*100
    plt.bar(np.arange(len(scree))+1, scree)
    plt.plot(np.arange(len(scree))+1, scree.cumsum(),c="red",marker='o')
    plt.xlabel("rang de l'axe d'inertie")
    plt.ylabel("pourcentage d'inertie")
    plt.title("Eboulis des valeurs propres")
    plt.show(block=False)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import numpy as np
import pandas as pd

def display_circles(n_comp, pca, axis_ranks, labels=None, label_rotation=0, lims=None):
    '''
    représente les axes sur le cercle des correlations
    '''
    for d1, d2 in axis_ranks: # On affiche les 3 premiers plans factoriels, donc les 6 premières composantes
        if d2 < n_comp:

            # initialisation de la figure
            fig, ax = plt.subplots(figsize=(7,6))

            # détermination des limites du graphique
            if lims is not None :
                xmin, xmax, ymin, ymax = lims
            elif pca.components_.shape[1] < 30 :
                xmin, xmax, ymin, ymax = -1, 1, -1, 1
            else :
                xmin, xmax, ymin, ymax = min(pca.components_[d1,:]), max(pca.components_[d1,:]), min(pca.components_[d2,:]), max(pca.components_[d2,:])

            # affichage des flèches
            # s'il y a plus de 30 flèches, on n'affiche pas le triangle à leur extrémité
            if pca.components_.shape[1] < 30 :
                plt.quiver(np.zeros(pca.components_.shape[1]), np.zeros(pca.components_.shape[1]),
                   pca.components_[d1,:], pca.components_[d2,:], 
                   angles='xy', scale_units='xy', scale=1, color="grey")
                # (voir la doc : https://matplotlib.org/api/_as_gen/matplotlib.pyplot.quiver.html)
            else:
                lines = [[[0,0],[x,y]] for x,y in pca.components_[[d1,d2]].T]
                ax.add_collection(LineCollection(lines, axes=ax, alpha=.1, color='black'))
            
            # affichage des noms des variables  
            if labels is not None:  
                for i,(x, y) in enumerate(pca.components_[[d1,d2]].T):
                    if x >= xmin and x <= xmax and y >= ymin and y <= ymax :
                        plt.text(x, y, labels[i], fontsize='14', ha='center', va='center', rotation=label_rotation, color="blue", alpha=0.5)
            
            # affichage du cercle
            circle = plt.Circle((0,0), 1, facecolor='none', edgecolor='b')
            plt.gca().add_artist(circle)

            # définition des limites du graphique
            plt.xlim(xmin, xmax)
            plt.ylim(ymin, ymax)
        
            # affichage des lignes horizontales et verticales
            plt.plot([-1, 1], [0, 0], color='grey', ls='--')
            plt.plot([0, 0], [-1, 1], color='grey', ls='--')

            # nom des axes, avec le pourcentage d'inertie expliqué
            plt.xlabel('F{} ({}%)'.format(d1+1, round(100*pca.explained_variance_ratio_[d1],1)))
            plt.ylabel('F{} ({}%)'.format(d2+1, round(100*pca.explained_variance_ratio_[d2],1)))

            plt.title("Cercle des corrélations (F{} et F{})".format(d1+1, d2+1))
            plt.show(block=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def display_circles_3d(n_comp, pca, axis_ranks, labels=None, label_rotation=0, lims=None):
    '''
    représente les axes sur le cercle des correlations
    '''
    for d1, d2, d3 in axis_ranks: # On affiche les 3 premiers plans factoriels, donc les 6 premières composantes
        if d3 < n_comp:

            # initialisation de la figure
            fig, ax = plt.subplots(figsize=(8,8))
            ax = plt.axes(projection='3d')

            # détermination des limites du graphique
            if lims is not None :
                xmin, xmax, ymin, ymax, zmin, zmax = lims
            elif pca.components_.shape[1] < 30 :
                xmin, xmax, ymin, ymax, zmin, zmax = -1, 1, -1, 1, -1, 1
            else :
                xmin, xmax, ymin, ymax, zmin, zmax = min(pca.components_[d1,:]), max(pca.components_[d1,:]), min(pca.components_[d2,:]), max(pca.components_[d2,:]), min(pca.components_[d3,:]), max(pca.components_[d3,:])

            # affichage des flèches
            lines = [[[0,0,0,x,y,z]] for x,y,z in pca.components_[[d1,d2,d3]].T]
            X, Y, Z, U, V, W = np.array(lines).T.tolist()
            ax.quiver(X, Y, Z, U, V, W)
            
            # affichage des noms des variables  
            if labels is not None:  
                for i,(x, y, z) in enumerate(pca.components_[[d1,d2,d3]].T):
                    if x >= xmin and x <= xmax and y >= ymin and y <= ymax and z >= zmin and z <= zmax :
                        ax.text(x, y, z, labels[i], fontsize='12', ha='center', va='center', rotation=label_rotation, color="blue", alpha=0.5)
            

            # définition des limites du graphique
            ax.set_xlim(xmin, xmax)
            ax.set_ylim(ymin, ymax)
            ax.set_zlim(zmin, zmax)
        
            # affichage des lignes horizontales et verticales
            #plt.plot([-1, 1], [0, 0], color='grey', ls='--')
            #plt.plot([0, 0], [-1, 1], color='grey', ls='--')

            # nom des axes, avec le pourcentage d'inertie expliqué
            ax.set_xlabel('F{} ({}%)'.format(d1+1, round(100*pca.explained_variance_ratio_[d1],1)))
            ax.set_ylabel('F{} ({}%)'.format(d2+1, round(100*pca.explained_variance_ratio_[d2],1)))
            ax.set_zlabel('F{} ({}%)'.format(d3+1, round(100*pca.explained_variance_ratio_[d3],1)))

            plt.title("Cercle des corrélations (F{}, F{} et F{})".format(d1+1, d2+1, d3+1))
            plt.show(block=False)

### Clustering

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import dendrogram

def plot_dendrogram(Z, names):
    plt.figure(figsize=(10,25))
    plt.title('Hierarchical Clustering Dendrogram')
    plt.xlabel('distance')
    dendrogram(
        Z,
        labels = names,
        orientation = "left",
    )
    plt.show()